In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
N = 20000

In [3]:
ip = np.random.rand(N) > 0.5
ip = ip.astype(int)


In [4]:
s = 2 * ip - 1

In [5]:
Es = 1

In [6]:
eb_no_db = np.arange(0,25)

In [7]:
ber_RxMF_EPA = np.zeros(len(eb_no_db))

ber_RxZF_EPA = np.zeros(len(eb_no_db))

ber_RxMMSE_EPA = np.zeros(len(eb_no_db))

ber_RxMF_SVD_P = np.zeros(len(eb_no_db))

ber_RxZF_SVD_P = np.zeros(len(eb_no_db))

In [8]:
def calculate_ber(eb_no_db,nTx,nRx,jj,Rz_inverse):
    
    
    equalize_bits = []
    
    for ii in range(N//2):
        
        H = (1/np.sqrt(2)) * (np.random.randn(nRx,nTx) + 1j * np.random.randn(nRx,nTx))
        
        n = (1/np.sqrt(2)) * (np.random.randn(2,1) + 1j * np.random.randn(2,1))
        
        x1 = s[2 * ii]
        
        x2 = s[2 * ii + 1]
        
        x =np.array([[x1],[x2]])
        
        
        
        Ns = np.linalg.matrix_rank(H)
        
        P = np.sqrt(Es / Ns) * np.vstack([
               np.eye(Ns),                
             np.zeros((nTx-Ns, Ns))           
          ])
        
        
        
        y = H @ P @ x + n * (10 ** (-eb_no_db[jj] / 20))
        
        
        H_hermitian = np.conjugate(H)
        
        H_hermitian = np.transpose(H_hermitian)
        
        P_hermitian = np.conjugate(P)
        
        P_hermitian = np.transpose(P_hermitian)
        
        W = P_hermitian @ H_hermitian @ Rz_inverse
        
        x_hat = np.dot(W,y)
        
        x_hat = np.real(x_hat > 0).astype(int)
        
        equalize_bits.append(x_hat[0,0])
        
        equalize_bits.append(x_hat[1,0])
        
    nErr = np.sum(ip != equalize_bits)
    
    return nErr
         

In [9]:
def calculate_ber_ZF(eb_no_db,nTx,nRx,jj,Rz_inverse):
    
    
    equalize_bits = []
    
    for ii in range(N//2):
        
        H = (1/np.sqrt(2)) * (np.random.randn(nRx,nTx) + 1j * np.random.randn(nRx,nTx))
        
        n = (1/np.sqrt(2)) * (np.random.randn(2,1) + 1j * np.random.randn(2,1))
        
        x1 = s[2 * ii]
        
        x2 = s[2 * ii + 1]
        
        x =np.array([[x1],[x2]])
        
        
        
        Ns = np.linalg.matrix_rank(H)
        
        P = np.sqrt(Es / Ns) * np.vstack([
               np.eye(Ns),                
             np.zeros((nTx-Ns, Ns))           
          ])
        
        
        
        y = H @ P @ x + n * (10 ** (-eb_no_db[jj] / 20))
        
        
        H_hermitian = np.conjugate(H)
        
        H_hermitian = np.transpose(H_hermitian)
        
        P_hermitian = np.conjugate(P)
        
        P_hermitian = np.transpose(P_hermitian)
        
        W = np.linalg.inv(P_hermitian @ H_hermitian @ Rz_inverse @ H @ P) 
        
        W = W @ P_hermitian @ H_hermitian @ Rz_inverse
        
        x_hat = np.dot(W,y)
        
        x_hat = np.real(x_hat > 0).astype(int)
        
        equalize_bits.append(x_hat[0,0])
        
        equalize_bits.append(x_hat[1,0])
        
    nErr = np.sum(ip != equalize_bits)
    
    return nErr
    
    

In [10]:
def calculate_ber_MMSE(eb_no_db,nTx,nRx,jj,Rz,Rz_inverse):
    
    
    equalize_bits = []
    
    for ii in range(N//2):
        
        H = (1/np.sqrt(2)) * (np.random.randn(nRx,nTx) + 1j * np.random.randn(nRx,nTx))
        
        n = (1/np.sqrt(2)) * (np.random.randn(2,1) + 1j * np.random.randn(2,1))
        
        x1 = s[2 * ii]
        
        x2 = s[2 * ii + 1]
        
        x =np.array([[x1],[x2]])
        
        
        
        Ns = np.linalg.matrix_rank(H)
        
        P = np.sqrt(Es / Ns) * np.vstack([
               np.eye(Ns),                
             np.zeros((nTx-Ns, Ns))           
          ])
        
        
        
        y = H @ P @ x + n * (10 ** (-eb_no_db[jj] / 20))
        
        
        H_hermitian = np.conjugate(H)
        
        H_hermitian = np.transpose(H_hermitian)
        
        P_hermitian = np.conjugate(P)
        
        P_hermitian = np.transpose(P_hermitian)
        
        W = np.linalg.inv(H @ P @ P_hermitian @ H_hermitian + Rz ) 
        
        W = P_hermitian @ H_hermitian @ W
        
        x_hat = np.dot(W,y)
        
        x_hat = np.real(x_hat > 0).astype(int)
        
        equalize_bits.append(x_hat[0,0])
        
        equalize_bits.append(x_hat[1,0])
        
    nErr = np.sum(ip != equalize_bits)
    
    return nErr

In [11]:
def RxMF():
    
    
    for jj in range(len(eb_no_db)):
        
        nTx = 2
        
        nRx = 2
        
        snr = 10**(eb_no_db[jj] / 10)
        
        noise_variance = (1/snr)
        
        Rz = noise_variance * np.eye(nRx)
        
        Rz_inverse = np.linalg.inv(Rz)
        
        bit_error = calculate_ber(eb_no_db,nTx,nRx,jj,Rz_inverse)
        
        ber_RxZF_EPA[jj] = bit_error / N
        
        
    return ber_RxZF_EPA
        
        
        

In [12]:
def RxZF():
    
    for jj in range(len(eb_no_db)):
        
        nTx = 2
        
        nRx = 2
        
        snr = 10**(eb_no_db[jj] / 10)
        
        noise_variance = (1/snr)
        
        Rz = noise_variance * np.eye(nRx)
        
        Rz_inverse = np.linalg.inv(Rz)
        
        bit_error = calculate_ber_ZF(eb_no_db,nTx,nRx,jj,Rz_inverse)
        
        ber_RxMF_EPA[jj] = bit_error / N
        
        
    return ber_RxMF_EPA
        
        

In [13]:
def RxMMSE():
    
    for jj in range(len(eb_no_db)):
        
        nTx = 2
        
        nRx = 2
        
        snr = 10**(eb_no_db[jj] / 10)
        
        noise_variance = (1/snr)
        
        Rz = noise_variance * np.eye(nRx)
        
        Rz_inverse = np.linalg.inv(Rz)
        
        bit_error = calculate_ber_MMSE(eb_no_db,nTx,nRx,jj,Rz,Rz_inverse)
        
        ber_RxMMSE_EPA[jj] = bit_error / N
        
        
    return ber_RxMMSE_EPA
    

In [14]:
def sigma_two(noise_variance,S1,S2,q_optimal_1):
    
    S = [ S1 , S2 ]
    
    q_optimal_1 = q_optimal_1

    sigma_two_q  = []
    
    summation_part = 0
    
    for kk in range(len(S)):
        
        summation_part = summation_part +  (noise_variance/np.square(S[kk]))
        
    mu = (1/len(S))* (Es + summation_part)
    
    for mm in range(len(S)):
        
        qi = mu - (noise_variance/np.square(S[mm]))
        
        if(qi > 0):
            
            sigma_two_q.append(qi)
            
        else:
            sigma_two_q = []
            
            sigma_two_q = q_optimal_1.copy()
            
            break
            
    return sigma_two_q

In [15]:
def sigma_one(noise_variance,S1):
    
    
    S = [S1]
    
    
    sigma_one_q = []
    
    summation_part = noise_variance / np.square(S1)
    
    mu = (1/len(S)) * (Es + summation_part )
    
    q_optimal = mu - (noise_variance / np.square(S1))
    
    if q_optimal > 0 :
        
        sigma_one_q.append(q_optimal)
        
    
    
    return sigma_one_q

In [16]:
def diagonalQ(S,noise_variance):
    
    
    S1 = S[0]
    
    S2 = S[1]
    
    S = [S1, S2]
    
    q_optimal_1 = sigma_one(noise_variance,S1)
    
    q_optimal_2 = sigma_two(noise_variance,S1,S2,q_optimal_1)
    
    return q_optimal_2
    

In [17]:
def design_precoder(V,S,noise_variance,nTx):
    
    Q = diagonalQ(S,noise_variance)
    
    Ns = max(2, len(Q))

    q_sqrt = np.sqrt(Q)
        
    q_sqrt = q_sqrt[:nTx]
        
    q_sqrt = np.pad(q_sqrt, (0, nTx - len(q_sqrt)), mode='constant', constant_values=0)
        
    Q_matrix = np.diag(q_sqrt)
    
    
    precoder = V[:,0:Ns] @ Q_matrix[0:Ns,0:Ns]
    
    return precoder

In [18]:
def calculate_ber_RxMF_SVD_P(noise_variance,jj,nTx,nRx,Rz,Rz_inverse):
    
    equalize_bits = []
    
    for ii in range(N//2):
        
        H = (1/np.sqrt(2)) * (np.random.randn(nRx,nTx) + 1j * np.random.randn(nRx,nTx))
        
        H_hermitian = np.conjugate(H)
        
        H_hermitian = np.transpose(H_hermitian)
        
        U,S,Vh = np.linalg.svd(H, full_matrices = False)
        
        V = np.conjugate(Vh)
        
        V = np.transpose(V)
        
        n = (1/np.sqrt(2)) * (np.random.randn(2,1) + 1j * np.random.randn(2,1))
        
        x1 = s[2 * ii]
        
        x2 = s[2 * ii + 1]
        
        x =np.array([[x1],[x2]])

        
        P = design_precoder(V,S,noise_variance,nTx)
        
        P_hermitian = np.conjugate(P)
        
        P_hermitian = np.transpose(P_hermitian)
        
        y = H @ P @ x + n * (10 ** (-eb_no_db[jj] / 20))
        
        W = P_hermitian @ H_hermitian @ Rz_inverse
        
        x_hat = np.dot(W,y)
        
        x_hat = np.real(x_hat > 0).astype(int)
        
        equalize_bits.append(x_hat[0,0])
        
        equalize_bits.append(x_hat[1,0])
        
    nErr = np.sum(ip != equalize_bits)
    
    return nErr

In [19]:
def RxMF_SVD_P():
    
    for jj in range(len(eb_no_db)):
        
        snr = 10**(eb_no_db[jj] / 10)
         
        noise_variance = (1/snr)
        
        nTx = 2
        
        nRx = 2
        
        Rz = noise_variance * np.eye(nRx)
        
        Rz_inverse = np.linalg.inv(Rz)
        
        bit_error = calculate_ber_RxMF_SVD_P(noise_variance,jj,nTx,nRx,Rz,Rz_inverse)
        
        ber_RxMF_SVD_P[jj] = bit_error / N
        
    return ber_RxMF_SVD_P
        

In [20]:
def calculate_ber_RxZF_SVD_P(noise_variance,jj,nTx,nRx,Rz,Rz_inverse):
    
    equalize_bits = []
    
    for ii in range(N//2):
        
        H = (1/np.sqrt(2)) * (np.random.randn(nRx,nTx) + 1j * np.random.randn(nRx,nTx))
        
        H_hermitian = np.conjugate(H)
        
        H_hermitian = np.transpose(H_hermitian)
        
        U,S,Vh = np.linalg.svd(H, full_matrices = False)
        
        V = np.conjugate(Vh)
        
        V = np.transpose(V)
        
        n = (1/np.sqrt(2)) * (np.random.randn(2,1) + 1j * np.random.randn(2,1))
        
        x1 = s[2 * ii]
        
        x2 = s[2 * ii + 1]
        
        x =np.array([[x1],[x2]])

        
        P = design_precoder(V,S,noise_variance,nTx)
        
        P_hermitian = np.conjugate(P)
        
        P_hermitian = np.transpose(P_hermitian)
        
        y = H @ P @ x + n * (10 ** (-eb_no_db[jj] / 20))
        
        W = np.linalg.inv(P_hermitian @ H_hermitian @ Rz_inverse @ H @ P)
        
        W = W @ P_hermitian @ H_hermitian @ Rz_inverse
        
        x_hat = np.dot(W,y)
        
        x_hat = np.real(x_hat > 0).astype(int)
        
        equalize_bits.append(x_hat[0,0])
        
        equalize_bits.append(x_hat[1,0])
        
    nErr = np.sum(ip != equalize_bits)
    
    return nErr

In [21]:
def RxZF_SVD_P():
    
    for jj in range(len(eb_no_db)):
        
        snr = 10**(eb_no_db[jj] / 10)
         
        noise_variance = (1/snr)
        
        nTx = 2
        
        nRx = 2
        
        Rz = noise_variance * np.eye(nRx)
        
        Rz_inverse = np.linalg.inv(Rz)
        
        bit_error = calculate_ber_RxZF_SVD_P(noise_variance,jj,nTx,nRx,Rz,Rz_inverse)
        
        ber_RxZF_SVD_P[jj] = bit_error / N
        
    return ber_RxZF_SVD_P

In [22]:
rxmf_epa = RxMF()

rxzf_epa = RxZF()

rxmmse_epa = RxMMSE()

rxmf_svd = RxMF_SVD_P()

rxzf_svd = RxZF_SVD_P()

plt.figure()

plt.semilogy(eb_no_db, rxmf_epa, 'm|-', linewidth=2, markersize=8, label='nTx=2 nRx=2 (mimo w/o csit RxMF)(EPA_P)')

plt.semilogy(eb_no_db, rxzf_epa, 'r*-', linewidth=2, markersize=8, label='nTx=2 nRx=2 (mimo w/o csit RxZF)(EPA_P)')

plt.semilogy(eb_no_db, rxmmse_epa, 'rv-', linewidth=2, markersize=8, label='nTx=2 nRx=2 (mimo w/o csit RxMMSE)(EPA_P)')

plt.semilogy(eb_no_db, rxmf_svd, 'm^-', linewidth=2, markersize=8, label='nTx=2 nRx=2 (mimo w/o csit RxMF)(EPA SVD P)')

plt.semilogy(eb_no_db, rxzf_svd, 'm>-', linewidth=2, markersize=8, label='nTx=2 nRx=2 (mimo w/o csit RxZF)(EPA SVD P)')

plt.grid(True)
plt.legend()
plt.xlabel('Eb/No, dB')
plt.ylabel('Bit Error Rate')
plt.title('SNR-dB')
plt.show()

LinAlgError: Singular matrix